In [109]:
from typing_extensions import TypedDict,List
from langgraph.graph.state import StateGraph, START

In [115]:
from numpy import add


class SubgraphState(TypedDict):
    a: str  
    b: str  
    f: str
    arr: List[int]

def subgraph_node_1(state: SubgraphState):
    return {"b": "biswas",}

def subgraph_node_2(state: SubgraphState):
    return {"f": state["a"]+ state['b'] }

def add_in_arr(state: SubgraphState):
    arr = state.get("arr", [])
    arr.append(1)
    return {"arr": arr}

subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node(subgraph_node_1)
subgraph_builder.add_node(subgraph_node_2)
subgraph_builder.add_node(add_in_arr)
subgraph_builder.add_edge(START, "subgraph_node_1")
subgraph_builder.add_edge("subgraph_node_1", "subgraph_node_2")
subgraph_builder.add_edge("subgraph_node_2", "add_in_arr")
subgraph = subgraph_builder.compile()

class ParentState(TypedDict):
    a: str
    b: str
    f: str
    arr: List[int]

def node_1(state: ParentState):
    return {"a": "hi! " + state["a"]}

builder = StateGraph(ParentState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", subgraph)
builder.add_node("node_3",add_in_arr)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", "node_3")
graph = builder.compile()

In [116]:
for chunk in graph.stream({"a": "anik"}):
    print(chunk)

{'node_1': {'a': 'hi! anik'}}
{'node_2': {'a': 'hi! anik', 'b': 'biswas', 'f': 'hi! anikbiswas', 'arr': [1]}}
{'node_3': {'arr': [1, 1]}}


In [117]:
graph.invoke({"a": "anik"})

{'a': 'hi! anik', 'b': 'biswas', 'f': 'hi! anikbiswas', 'arr': [1, 1]}